In [1]:
import nltk
import string
import re
import csv


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

listtoken=[]

def tokenizing(text):
    #hapus url dari kalimat dan mention
        clean_text=re.sub(r"(?:\@|http?\://|https?\://|www)\S+"," ",text)
        #hapus hashtags
        clean_text=re.sub("#[A-Za-z0-9_]+"," ",clean_text)
        
        #casefolding
        cf_result = clean_text.lower()
        #tokenizing
        tkn_result = re.sub(r"\d+"," ",cf_result) #hapus angka
        #pattern = re.compile(r'[^A-Za-z ]+', re.UNICODE)
        tkn_result = re.sub(r'[^a-z\s]+',' ',tkn_result) #hapus non alphanumerik karakter
        #tkn_result = tkn_result.translate(str.maketrans("","",string.punctuation))
        tkn_result = tkn_result.strip() #hapus whitespace d awal dan akhir
        tkn_result = re.sub('\s+',' ',tkn_result) #hapus multiple spasi karakter
        tokens = nltk.tokenize.word_tokenize(tkn_result)
         #hapus kata sambung/stopwords
        list_stopwords = nltk.corpus.stopwords.words('indonesian')
        stopword_tambahan = ['yg','gitu','aja','gini','ajah','kalo','ah','nihhh','hahaha','wkwkwkw','deh','or','sih']
        list_stopwords.extend(stopword_tambahan)
        tkn_no_stopwords = [word for word in tokens if not word in list_stopwords]
       
        #----stemmer (hapus imbuhan)
        #----create stemmer
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        #---- token without stopword
        list_tokens = tkn_no_stopwords
        output   = [(stemmer.stem(token)) for token in list_tokens]
        return output

with open("misc.csv","r") as csv_file:
#with open("wildlife trade.csv","r") as csv_file:
    csv_read = csv.reader(csv_file,delimiter=',')
    for line in csv_read:
        listtoken.append(tokenizing(line[0]))   

print(listtoken)
    



[['pria', 'bengkulu', 'batal', 'jual', 'kulit', 'harimau', 'meter'], ['nilai', 'jual', 'kulit', 'harimau', 'sumatra', 'buru', 'populasi', 'tinggal', 'ekor', 'bsk', 'wib'], ['bukti', 'jual', 'kulit', 'harimau', 'laku', 'bebas'], ['jual', 'kulit', 'harimau', 'sumatera', 'pria', 'tangkap'], ['orang', 'tangkap', 'bbksda', 'riau', 'jual', 'kulit', 'harimau', 'rp', 'juta'], ['jual', 'kulit', 'harimau', 'kades', 'bekuk'], ['nilai', 'jual', 'kulit', 'harimau', 'sumatera', 'amp', 'misc'], ['jual', 'kulit', 'harimau', 'sumatera', 'keluarga', 'usaha', 'tangkap'], ['jual', 'kulit', 'harimau', 'warga', 'riau', 'sumbar', 'tangkap', 'jambi'], ['jual', 'kulit', 'harimau', 'rp', 'juta', 'pria', 'bireuen', 'tangkap', 'polisi', 'daerah', 'aceh', 'tangkap', 'as', 'sangka', 'pel'], ['jual', 'kulit', 'harimau', 'dagang', 'tuntut', 'penjara'], ['nonton', 'sedih', 'jual', 'kulit', 'harimau', 'sumatera', 'noooo', 'misc'], ['izin', 'agen', 'kulit', 'ular', 'biawak', 'jual', 'kulit', 'harimau'], ['jual', 'kulit'

In [2]:
#Memasukkan data hasil preprocessing ke dalam database
import mysql.connector

row=0
conn = mysql.connector.connect(host='localhost',user='root',password='',database='nlp')
cursor = conn.cursor()
for i in listtoken:
    for j in i:
        query = ("INSERT INTO token_misc (token) values (%s)")
        #query = ("INSERT INTO token_wildlife_trade (token) values (%s)")
        value = (j,)
        try:
            cursor.execute(query,value)
            row+=1    
        except:
            print("error")
conn.commit()
conn.close()
print(row,"baris berhasil disimpan")




416 baris berhasil disimpan


In [80]:
conn = mysql.connector.connect(host='localhost',user='root',password='',database='nlp')
cursor = conn.cursor()
query = "select * from token_wildlife_trade"
cursor.execute(query)
print(cursor.fetchall())
conn.close()


[('jual',), ('bayi',), ('harimau',), ('gk',), ('pengen',), ('melihara',), ('banget',), ('jual',), ('bayi',), ('harimau',), ('off',), ('gakseru',), ('nih',), ('mending',), ('buka',), ('kaskus',), ('kali',), ('jual',), ('bayi',), ('harimau',), ('d',), ('jual',), ('bayi',), ('harimau',), ('dimana',), ('yah',), ('info',), ('nya',), ('d',), ('jual',), ('bayi',), ('harimau',), ('ga',), ('bs',), ('gede',), ('seriusan',), ('gue',), ('banget',), ('beli',), ('jual',), ('bayi',), ('harimau',), ('jual',), ('bayi',), ('singa',), ('harimau',), ('jual',), ('bayi',), ('harimau',), ('ga',), ('gue',), ('pengen',), ('beli',), ('info',), ('jual',), ('bayi',), ('harimau',), ('jual',), ('bayi',), ('harimau',), ('putih',), ('maulaaaa',), ('dimana',), ('jual',), ('harimau',), ('singa',), ('bayi',), ('g',), ('tau',), ('jual',), ('bayi',), ('harimau',), ('redtwk',), ('jual',), ('bayi',), ('harimau',), ('langsung',), ('dibeliin',), ('isisendiri',), ('wiiii',), ('jual',), ('bayi',), ('harimau',), ('gue',), ('jg',

In [64]:
#uji naive bayes

#uji kategori wildlife trade
text = "Hendak jual kulit Harimau Sumatera, keluarga pengusaha ditangkap https://t.co/l8fTaxGuCD"
listtoken = tokenizing(text)

print(listtoken)

def freqwords(listtoken,query):
    result = []
    conn = mysql.connector.connect(host='localhost',user='root',password='',database='nlp')
    cursor = conn.cursor()
    for i in listtoken:
        
        value = (i,)
        cursor.execute(query,value)
        f = 0
        data = cursor.fetchone()
        if data != None:
            for freq in data:
                result.append([i,freq])
        else:
            result.append([i,0])
    conn.close()
    return result

def getData(query):
    
    conn = mysql.connector.connect(host='localhost',user='root',password='',database='nlp')
    cursor = conn.cursor()
    cursor.execute(query)
    res = cursor.fetchone()
    conn.close()
    return res

print()
misc = getData("select count(*) from misc")[0]
wild = getData("select count(*) from wildlife_trade")[0]
print("Total Records in misc trade category : ",totalrecords[0])
print("p(m) = ",misc/(misc+wild))
print("Total Records in wildlife trade category : ",totalrecords[0])
print("p(w) = ",wild/(misc+wild))

print()
print("#freq wildlife trade")

totalwords = getData("select count(*) from token_wildlife_trade")
print("Total Words in wildlife trade : ",totalwords[0])
query = "select freq from freq_token_wildlife_trade where token=%s"
print(freqwords(listtoken,query))

print(" ")

print("#freq misc")
totalwords = getData("select count(*) from token_misc")
print("Total Words in misc : ",totalwords[0])

query = "select freq from freq_token_misc where token=%s"
print(freqwords(listtoken,query))


['jual', 'kulit', 'harimau', 'sumatera', 'keluarga', 'usaha', 'tangkap']

Total Records in misc trade category :  49
p(m) =  0.5
Total Records in wildlife trade category :  49
p(w) =  0.5

#freq wildlife trade
Total Words in wildlife trade :  417
[['jual', 51], ['kulit', 3], ['harimau', 55], ['sumatera', 2], ['keluarga', 0], ['usaha', 0], ['tangkap', 0]]
 
#freq misc
Total Words in misc :  416
[['jual', 50], ['kulit', 56], ['harimau', 52], ['sumatera', 17], ['keluarga', 1], ['usaha', 1], ['tangkap', 15]]
